In [ ]:
import numpy as np
from itertools import groupby
from operator import itemgetter
import time
from google.colab import output
import sys

In [ ]:
def duplicates(x):
  return np.sum(x) != np.sum(np.unique(x))

def valid(sudoku):
  return not any((duplicates(sudoku[r,:]) for r in range(0, 9)) or (duplicates(sudoku[:,c]) for c in range(0, 9)) or (duplicates(sudoku[r:r+3,c:c+3]) for r in range(0, 9, 3) for c in range(0, 9, 3)) )
  
def is_solved(sudoku):
  return valid(sudoku) and sudoku.sum() == (45*9)

def print_sudoku(sudoku):
    print("+-------+-------+-------+")
    for b in range(0, 9, 3):
        for r in range(3):
            print("|", " | ".join(" ".join(str(_) for _ in sudoku[b+r, c:c+3]) for c in range(0, 9, 3)), "|")
        print("+-------+-------+-------+")

def possible_values(sudoku, r, c):
  rem = sudoku[r,:]
  rem = np.append(rem, sudoku[:, c])
  rem = np.append(rem, sudoku[(r//3)*3:(r//3)*3+3, (c//3)*3:(c//3)*3+3])
  return [value for value in range(1,10) if value not in rem]

def exclusive(map, r, c):
  grid = [(row, col) for row in range((r//3)*3,(r//3)*3+3) for col in range((c//3)*3, (c//3)*3+3) if (row, col) != (r, c) ]
  p1 = [t[1] for t in map if t[0] in grid]
  notBoxExclusive = np.unique([y for x in p1 for y in x])
  #print("notBoxExclusive = ", notBoxExclusive)
  p2 = [t[1] for t in map if (t[0] != (r, c)) and ( (t[0][0] == r) or (t[0][1] == c) ) ]
  NotColRowExclusive = np.unique([y for x in p2 for y in x])
  #print("NotColRowExclusive = ", NotColRowExclusive)
  allowed = [n for pos, val in map for n in val if pos == (r, c)]
  #print("allowed = ", allowed)
  ret = [value for value in range(1, 10) if (value in allowed) and ((value not in notBoxExclusive) or (value not in NotColRowExclusive) )]
  return ret

def purge(map) :
  newMap = []
  for pos, values in map :
    r = pos[0]
    c = pos[1]
    gridNotRow = [(row, col) for row in range((r//3)*3,(r//3)*3+3) for col in range((c//3)*3, (c//3)*3+3) if row != r ]
    p1 = [t[1] for t in map if t[0] in gridNotRow] 
    notBoxRowExclusive = np.unique([y for x in p1 for y in x])
    #print("notBoxRowExclusive = ", notBoxRowExclusive)
    gridNotCol = [(row, col) for row in range((r//3)*3,(r//3)*3+3) for col in range((c//3)*3, (c//3)*3+3) if col != c ]
    p2 = [t[1] for t in map if t[0] in gridNotCol]
    notBoxColExclusive = np.unique([y for x in p2 for y in x])
    #print("notBoxColExclusive = ", notBoxColExclusive)
    allowed = [n for pos, val in map for n in val if pos == (r, c)]
    #print("allowed = ", allowed)
    retRow = [value for value in range(1, 10) if (value in allowed) and (value not in notBoxRowExclusive)]
    retCol = [value for value in range(1, 10) if (value in allowed) and (value not in notBoxColExclusive)]
    #print("retRow = ", retRow)
    #print("retCol = ", retCol)
    #print(map)
    for pos, values in map :
      if(pos[0] == r and pos[1] not in range((c//3)*3, (c//3)*3+3)) :
        try:
          values = [ v for v in values if v not in retRow]
        except ValueError:
          pass
      if(pos[0] not in range((r//3)*3,(r//3)*3+3) and pos[1] == c) :
        try:
          values = [ v for v in values if v not in retCol]
        except ValueError:
          pass
      newMap.append((pos, values))
    #print(newMap)
  return newMap

def collisions(oners):
  #print(oners)
  for pos, values in oners :
    #print(pos)
    r = pos[0]
    c = pos[1]
    gridSameRow = [(pos[0], col) for col in range((c//3)*3, (c//3)*3+3) if col != c]
    p1 = [n for t in oners for n in t[1] if t[0] in gridSameRow]
    #print("p1 = ", p1)
    gridSameCol = [(row, pos[1]) for row in range((r//3)*3, (r//3)*3+3) if row != r]
    p2 = [n for t in oners for n in t[1] if t[0] in gridSameCol]
    #print("p2 = ", p2)
    if(values[0] not in p1 and values[0] not in p2):
      return False
  #print(newOners)
  return True

In [ ]:
def solve(sudoku):
  #output.clear()
  #print(f"{sudoku}")
  #sys.stdout.flush()
  global counter, stop
  counter += 1
  map = sorted(((k, list(list(zip(*g))[2])) for k, g in groupby([(r, c, n) for r in range(9) for c in range(9) for n in possible_values(sudoku, r, c) if sudoku[r,c] == 0], itemgetter(0, 1))), key=lambda t: len(t[1]))
  l = len(map)
  p =  np.count_nonzero(sudoku == 0)
  #print(f"len(map) = {l}, noz_zero = {p}")
  if l == 0 or p == 0 or l != p :
    #print("Dead end!")
    #print(f"{sudoku}")
    return 1
  oners = [o for o in map if len(o[1]) == 1]
  if(oners and collisions(oners)):
    return
  else :
    for pos, values in oners :
      sudoku[pos] = values[0]
  if(not valid(sudoku)):
    #print(f"Invalid sudoku")
    #print(oners)
    return 2
  elif (0 not in sudoku and is_solved(sudoku)):
    print(f"Solution found at {counter}-th iteration:")
    print_sudoku(sudoku)
    stop = True
    return
  elif len(oners) != 0:
    solve(sudoku)
    if (stop):
      return
  else : 
    map2 = ((k, list(list(zip(*g))[2])) for k, g in groupby([(pos[0], pos[1], rem) for pos, values in map for rem in exclusive(map, pos[0], pos[1])], itemgetter(0, 1)))
    map2 = [o for o in map2 if len(o[1]) == 1]
    #print(map2)
    if(len(map2) != 0) :
      for pos, values in map2 :
        sudoku[pos] = values[0]
      solve(sudoku)
      if (stop):
        return
    else :
    #  print(map2)
      map = purge(map)
      #map = [(pos, n) for pos, values in map for n in values]
      for pos, values in map :
        for n in values :
          sudoku[pos] = n
          if (valid(sudoku)) :
            r = solve(sudoku)
            if(r == 1) :
              #print("Dead end!")
              sudoku[pos] = 0
              #break
            elif(r == 2) :
              #print("Invalid sudoku!")
              sudoku[pos] = 0
              #print("sudoku valid = ", valid(sudoku), r)
              return
            if (stop):
              return
          else :
            sudoku[pos] = 0

In [ ]:
## Test on Simple Sudoku, very fast
sudoku = np.array([[6, 0, 4,    0, 7, 0,    0, 0, 1],
                   [0, 5, 0,    0, 0, 0,    0, 7, 0], 
                   [7, 0, 0,    5, 9, 6,    8, 3, 4], 
 
                   [0, 8, 0,    0, 0, 2,    4, 9, 0], 
                   [1, 0, 0,    0, 0, 0,    0, 0, 3], 
                   [0, 6, 9,    7, 0, 0,    0, 5, 0], 
 
                   [9, 1, 8,    3, 6, 7,    0, 0, 5], 
                   [0, 4, 0,    0, 0, 0,    0, 6, 0], 
                   [2, 0, 0,    0, 5, 0,    7, 0, 8]], dtype=np.int8)

counter = 0
stop = False

t1 = time.perf_counter()
solve(sudoku)
t2 = time.perf_counter()
print("Time taken to run:", t2-t1, "seconds")

[((0, 7), [2]), ((2, 1), [2]), ((5, 8), [2]), ((6, 6), [2]), ((8, 1), [3]), ((0, 3), [2, 8]), ((0, 5), [3, 8]), ((1, 0), [3, 8]), ((2, 2), [1, 2]), ((3, 0), [3, 5]), ((3, 3), [1, 6]), ((3, 4), [1, 3]), ((3, 8), [6, 7]), ((4, 1), [2, 7]), ((4, 4), [4, 8]), ((4, 6), [2, 6]), ((4, 7), [2, 8]), ((5, 0), [3, 4]), ((5, 6), [1, 2]), ((6, 7), [2, 4]), ((7, 0), [3, 5]), ((7, 8), [2, 9]), ((8, 2), [3, 6]), ((8, 7), [1, 4]), ((0, 1), [2, 3, 9]), ((0, 6), [2, 5, 9]), ((1, 2), [1, 2, 3]), ((1, 6), [2, 6, 9]), ((1, 8), [2, 6, 9]), ((3, 2), [3, 5, 7]), ((4, 2), [2, 5, 7]), ((7, 2), [3, 5, 7]), ((7, 4), [1, 2, 8]), ((7, 5), [1, 8, 9]), ((8, 3), [1, 4, 9]), ((8, 5), [1, 4, 9]), ((1, 3), [1, 2, 4, 8]), ((1, 5), [1, 3, 4, 8]), ((4, 3), [4, 6, 8, 9]), ((4, 5), [4, 5, 8, 9]), ((5, 4), [1, 3, 4, 8]), ((5, 5), [1, 3, 4, 8]), ((7, 3), [1, 2, 8, 9]), ((7, 6), [1, 2, 3, 9]), ((1, 4), [1, 2, 3, 4, 8])]
[((0, 1), [9]), ((0, 3), [8]), ((2, 2), [1]), ((4, 1), [7]), ((4, 6), [6]), ((4, 7), [8]), ((5, 6), [1]), ((6, 

In [ ]:
## Imported for compatibility reasons
def _contains_duplicates(X):
    return np.sum(np.unique(X)) != np.sum(X)

def contains_duplicates(sol):
    return any(_contains_duplicates(sol[r,:]) for r in range(9)) or \
           any(_contains_duplicates(sol[:,r]) for r in range(9)) or \
           any(_contains_duplicates(sol[r:r+3:,c:c+3]) for r in range(0,9,3) for c in range(0,9,3))

def sudoku_generator(sudokus=1, *, kappa=5, random_seed=None):
    if random_seed:
        np.random.seed(random_seed)
    for puzzle in range(sudokus):
        sudoku = np.zeros((9, 9), dtype=np.int8)
        for cell in range(np.random.randint(kappa)):
            for p, val in zip(np.random.randint(0, 8, size=(9, 2)), range(1, 10)):
                tmp = sudoku.copy()
                sudoku[tuple(p)] = val
                if contains_duplicates(sudoku):
                    sudoku = tmp
        yield sudoku.copy()

In [ ]:
for sudoku in sudoku_generator(sudokus=1, random_seed=42):
    counter = 0
    stop = False
    print_sudoku(sudoku)
    print("Number of clues = ", np.count_nonzero(sudoku))
    t1 = time.perf_counter()
    solve(sudoku)
    t2 = time.perf_counter()
    print("Time taken to run:", t2-t1, "seconds")

+-------+-------+-------+
| 0 0 1 | 0 0 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 0 3 0 |
| 0 0 6 | 8 0 0 | 5 2 0 |
+-------+-------+-------+
| 9 7 0 | 4 0 0 | 0 8 0 |
| 6 0 0 | 0 3 0 | 1 0 0 |
| 0 0 0 | 0 8 6 | 0 0 0 |
+-------+-------+-------+
| 0 4 0 | 9 0 0 | 0 0 0 |
| 0 0 9 | 5 7 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 0 0 0 |
+-------+-------+-------+
Number of clues =  20
Solution found at 126-th iteration:
+-------+-------+-------+
| 7 9 1 | 3 2 5 | 8 6 4 |
| 5 2 8 | 6 9 4 | 7 3 1 |
| 4 3 6 | 8 1 7 | 5 2 9 |
+-------+-------+-------+
| 9 7 2 | 4 5 1 | 3 8 6 |
| 6 8 4 | 2 3 9 | 1 5 7 |
| 1 5 3 | 7 8 6 | 4 9 2 |
+-------+-------+-------+
| 3 4 7 | 9 6 8 | 2 1 5 |
| 2 1 9 | 5 7 3 | 6 4 8 |
| 8 6 5 | 1 4 2 | 9 7 3 |
+-------+-------+-------+
Time taken to run: 1.1370337390000032 seconds


In [ ]:
## Validation Test
sudoku = np.array([[0, 7, 0,   4, 0, 5,   0, 0, 9],
                   [0, 0, 0,   0, 2, 0,   4, 0, 0], 
                   [0, 0, 5,   0, 6, 0,   0, 0, 0], 
  
                   [0, 0, 0,   0, 5, 0,   0, 0, 0], 
                   [0, 0, 7,   3, 0, 1,   6, 0, 0], 
                   [0, 9, 0,   0, 0, 0,   0, 8, 0], 
                    
                   [0, 0, 0,   2, 0, 0,   0, 0, 0], 
                   [3, 0, 0,   0, 0, 0,   0, 0, 6], 
                   [0, 0, 1,   7, 0, 4,   3, 0, 0]], dtype=np.int8)

counter = 0
stop = False
t1 = time.perf_counter()
solve(sudoku)
t2 = time.perf_counter()
print("Time taken to run:", t2-t1, "seconds")

Solution found at 302-th iteration:
+-------+-------+-------+
| 1 7 3 | 4 8 5 | 2 6 9 |
| 9 8 6 | 1 2 7 | 4 3 5 |
| 4 2 5 | 9 6 3 | 8 1 7 |
+-------+-------+-------+
| 6 3 2 | 8 5 9 | 1 7 4 |
| 8 5 7 | 3 4 1 | 6 9 2 |
| 1 9 4 | 6 7 2 | 5 8 3 |
+-------+-------+-------+
| 7 4 8 | 2 3 6 | 9 5 1 |
| 3 2 9 | 5 1 8 | 7 4 6 |
| 5 6 1 | 7 9 4 | 3 2 8 |
+-------+-------+-------+
Time taken to run: 1.5060989360000008 seconds


In [ ]:
sudoku
counter


122